In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re

In [2]:
#기존의 크롤링 방ㅇ법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>83377</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;222677판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[양수금]]&gt;사건명&gt;&lt;사건번호&gt;2021그753사건번호&gt;&lt;선고일자&gt;2022.06.21선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;결정판결유형&gt;&lt;선고&gt;자선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=222677&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;222689판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[가처분이의]]&gt;사건명&gt;&lt;사건번호&gt;2019마6625사건번호&gt;&lt;선고일자&gt;2022.06.16선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;결정판결유형&gt;&lt;선고&gt;자선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=222689&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="3">&lt;판례일련번호&gt;2226

In [3]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [4]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

"for child in xtree:\n    print(':',child.tag,child.attrib)\nprint(xtree.find('page').text)"

In [5]:
xtree[3].text #totalCnt

'83377'

In [6]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
222677
양수금
2021그753
2022.06.21
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222677&type=HTML&mobileYn=


 {'id': '2'}
222689
가처분이의
2019마6625
2022.06.16
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222689&type=HTML&mobileYn=


 {'id': '3'}
222687
분양대금반환청구의소[휴양 콘도미니엄 수분양자가 분양계약상 고압선 지중화 의무가 불이행되었다는 이유로 계약을 해제하고 분양대반환 등을 구하고 있는 사안]
2022다203804
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222687&type=HTML&mobileYn=


 {'id': '4'}
222691
토지인도
2018다301350
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222691&type=HTML&mobileYn=


 {'id': '5'}
222685
재심기각결정에대한재항고
2022모509
2022.06.16
대법원
None
형사
400102
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222685&type=HTML&mobileYn=


 {'id': '6'}
222697
부당이득금
2022다211850
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222697&type=HTML&mobileYn=


 {'id': '7'}
2

In [7]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [8]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

 79%|███████▊  | 3274/4168 [09:53<02:41,  5.52it/s]


KeyboardInterrupt: 

In [9]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,222677,양수금,2021그753,2022.06.21,대법원,None,민사,400101,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,222689,가처분이의,2019마6625,2022.06.16,대법원,None,민사,400101,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,222687,분양대금반환청구의소[휴양 콘도미니엄 수분양자가 분양계약상 고압선 지중화 의무가 불이...,2022다203804,2022.06.16,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,222691,토지인도,2018다301350,2022.06.16,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,222685,재심기각결정에대한재항고,2022모509,2022.06.16,대법원,None,형사,400102,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [10]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [12]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['사건명', '사건번호', '선고일자','판결유형','판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
case_list=pd.DataFrame(columns=contents)

def remove_tag(content):
    cleaned=re.sub(',','$',re.sub('<.*?>', '', content))
    return cleaned

for pid,l in zip(prec_list['판례일련번호'],prec_list['판례상세링크']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row={}
    row['판례일련번호']=pid
    for content in contents:
        
        try:
            text=xtree.find(content).text
            text=remove_tag(text)
        except:
            text="-"
        row[content]=text
    case_list=pd.concat([case_list,pd.DataFrame([row],columns=row.keys())],axis=0)
case_list.reset_index(inplace=True)
        
        

        

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
case_list.head(5)

,index,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,법원이 반대급부의 이행과 동시에 금전 등 대체물이나 일정한 수량의 유가증권의 지...,법원은 금전 등 대체물이나 유가증권의 일정한 수량의 지급을 목적으로 하는 청구에...,민사소송법 제462조\n,-,【채권자$ 특별항고인】 주택도시보증공사 (소송대리인 법무법인 온새미로 담당변호사 김...,222677
1,0,[1] 불법행위로 인한 손해배상채무는 불법행위 당시부터 지연손해금이 발생하는지 ...,[1] 불법행위로 인한 손해배상채무는 손해발생과 동시에 이행기에 있는 것으로$ ...,[1] 민법 제393조$ 제750조$ 제763조 / [2] 민법 제393조$ 제7...,[1] 대법원 1975. 5. 27. 선고 74다1393 판결$ 대법원 1998....,【원고$ 피상고인 겸 상고인】 원고 (소송대리인 법무법인 대산 담당변호사 신지영 외...,222699
2,0,[1] 개인정보를 제공받은 자의 개인정보 보호법 제71조 제2호 위반죄는 정보제...,[1] 개인정보 보호법 제71조 제2호는 ‘제18조 제1항·제2항(제39조의14...,[1] 개인정보 보호법 제71조 제2호 / [2] 개인정보 보호법 제71조 제2호\n,-,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 박성빈【원심판결】 서울...,222683
3,0,[1] 민법 제321조에서 정한 유치권의 불가분성은 목적물이 분할 가능하거나 수...,"[1] 민법 제321조는 ""유치권자는 채권 전부의 변제를 받을 때까지 유치물 전...",[1] 민법 제321조$ 상법 제1조$ 제58조 / [2] 민법 제320조 제1항...,[1] 대법원 2007. 9. 7. 선고 2005다16942 판결(공2007하$ ...,【원고(탈퇴)】 유안타증권 주식회사【원고 승계참가인】 주식회사 금룡조경【원고 승계참...,222691
4,0,형사재판에서 범죄사실을 인정하기 위한 증거의 증명력 정도 / 법정형이 무거운 범...,형사재판에서 범죄사실의 인정은 법관으로 하여금 합리적인 의심을 할 여지가 없을 ...,형사소송법 제307조$ 제308조\n\n,대법원 2006. 3. 9. 선고 2005도8675 판결(공2006상$ 685)$ ...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 모병철【원심판결】 대구...,222679


In [ ]:
from pathlib import Path  
filepath = Path('case_list.csv')  
case_list.to_csv(filepath,index=False)

In [ ]:
case_list.head(10)

,index,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,법원이 반대급부의 이행과 동시에 금전 등 대체물이나 일정한 수량의 유가증권의 지...,법원은 금전 등 대체물이나 유가증권의 일정한 수량의 지급을 목적으로 하는 청구에...,민사소송법 제462조\n,-,【채권자$ 특별항고인】 주택도시보증공사 (소송대리인 법무법인 온새미로 담당변호사 김...,222677
1,0,[1] 불법행위로 인한 손해배상채무는 불법행위 당시부터 지연손해금이 발생하는지 ...,[1] 불법행위로 인한 손해배상채무는 손해발생과 동시에 이행기에 있는 것으로$ ...,[1] 민법 제393조$ 제750조$ 제763조 / [2] 민법 제393조$ 제7...,[1] 대법원 1975. 5. 27. 선고 74다1393 판결$ 대법원 1998....,【원고$ 피상고인 겸 상고인】 원고 (소송대리인 법무법인 대산 담당변호사 신지영 외...,222699
2,0,[1] 개인정보를 제공받은 자의 개인정보 보호법 제71조 제2호 위반죄는 정보제...,[1] 개인정보 보호법 제71조 제2호는 ‘제18조 제1항·제2항(제39조의14...,[1] 개인정보 보호법 제71조 제2호 / [2] 개인정보 보호법 제71조 제2호\n,-,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 박성빈【원심판결】 서울...,222683
3,0,[1] 민법 제321조에서 정한 유치권의 불가분성은 목적물이 분할 가능하거나 수...,"[1] 민법 제321조는 ""유치권자는 채권 전부의 변제를 받을 때까지 유치물 전...",[1] 민법 제321조$ 상법 제1조$ 제58조 / [2] 민법 제320조 제1항...,[1] 대법원 2007. 9. 7. 선고 2005다16942 판결(공2007하$ ...,【원고(탈퇴)】 유안타증권 주식회사【원고 승계참가인】 주식회사 금룡조경【원고 승계참...,222691
4,0,형사재판에서 범죄사실을 인정하기 위한 증거의 증명력 정도 / 법정형이 무거운 범...,형사재판에서 범죄사실의 인정은 법관으로 하여금 합리적인 의심을 할 여지가 없을 ...,형사소송법 제307조$ 제308조\n\n,대법원 2006. 3. 9. 선고 2005도8675 판결(공2006상$ 685)$ ...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 모병철【원심판결】 대구...,222679
5,0,예외적으로 유책배우자의 이혼청구를 허용할 수 있는 경우 및 그 판단 기준 / 이...,재판상 이혼원인에 관한 민법 제840조는 원칙적으로 유책주의를 채택하고 있는 것...,민법 제826조 제1항$ 제840조\n,대법원 2015. 5. 29. 선고 2013므2441 판결(공2015하$ 875)$...,【원고$ 상고인】 원고 (소송대리인 법무법인 동원 담당변호사 안창근 외 2인)【피고...,222695
6,0,[1] 학생에게 부여된 학습권이 학교의 설립·운영 주체 또는 학교교육의 단계에 ...,[1] 헌법은 모든 국민이 능력에 따라 균등하게 교육을 받을 권리를 가진다고 하...,[1] 헌법 제31조 제1항$ 교육기본법 제3조$ 제8조$ 제11조 제2항$ 제1...,[2] 대법원 2007. 9. 20. 선고 2005다25298 판결(공2007하$...,【원고$ 피상고인】 별지 원고 명단 기재와 같다. (소송대리인 법무법인 인터로 담당...,222693
7,0,채무자 회생 및 파산에 관한 법률 제119조 제1항에 따라 관리인이 쌍무계약을 해제...,회생절차폐지결정은 그 확정 시점이 회생계획 인가 이전 또는 이후인지에 관계없이 ...,채무자 회생 및 파산에 관한 법률 제119조 제1항$ 제286조$ 제288조 제1항...,-,【원고$ 상고인】 원고 (소송대리인 변호사 신건호 외 2인)【피고$ 피상고인】 주식...,222697
8,0,제1심 유죄판결이 항소 또는 상고기각판결로 확정된 경우$ 형벌조항에 대한 헌법재...,형벌에 관한 법률조항에 대하여 헌법재판소의 위헌결정이 선고되어 헌법재판소법 제4...,형사소송법 제420조 제1호$ 제2호$ 제7호$ 제421조$ 헌법재판소법 제47조 ...,대법원 2015. 10. 2. 자 2015재도75 결정$ 대법원 2022. 3. 1...,【피 고 인】 피고인【재항고인】 피고인【원심결정】 서울북부지법 2022. 2. 17...,222685
9,0,[1] 과거의 법률관계가 확인의 소의 대상이 될 수 있는 경우 [2] 확인의 ...,[1] 확인의 소는 원칙적으로 분쟁 당사자 사이의 권리 또는 법률상 지위에 현존...,[1] 민사소송법 제250조 / [2] 민사소송법 제134조[직권조사사항]$ 제1...,[1][2] 대법원 2020. 8. 20. 선고 2018다249148 판결(공20...,【원고$ 피상고인】 원고 (소송대리인 변호사 황규경)【피고$ 상고인】 주식회사 ○○...,222701
